In [1]:
import numpy as np
import cv2 as cv

* normaliseren per dataset (z-score vs 0-1)
* normaliseren per plaatje (z-score vs 0-1)
* converteren naar grijswaarden plaatje
* histogram test-set converteren naar histogram training set
* gabor filter
* normaliseren per cluster

In [2]:
from ProjectPaths import ProjectPaths

ProjectPaths.instance(r"/home/tdjg/Documents/DeepSolaris")

In [3]:
mask_images = np.load(ProjectPaths.instance().file_in_image_dir("ds_image_masks.npy"))

In [4]:
images = np.load(ProjectPaths.instance().file_in_image_dir("ds_images.npy"))
images[0].shape

(75, 75, 3)

In [5]:
def avg_image(image):
    avg_kernel = np.ones((3,3), dtype=np.float32)
    avg_kernel /= 9
    return cv.filter2D(image, -1, avg_kernel)

In [6]:
def window_locations(r = 2, n_channels = 3):
    x = [-r, 0, r]
    y = [-r, 0, r]
    mask = np.zeros((1+2*r,1+2*r))
    
    for xc in x:
        for yc in y:
            mask[r + yc, r + xc] = 1.0
    return np.tile(mask,(n_channels,1,1)).T
    
window_mask = window_locations()
window_mask

array([[[1., 1., 1.],
        [0., 0., 0.],
        [1., 1., 1.],
        [0., 0., 0.],
        [1., 1., 1.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[1., 1., 1.],
        [0., 0., 0.],
        [1., 1., 1.],
        [0., 0., 0.],
        [1., 1., 1.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[1., 1., 1.],
        [0., 0., 0.],
        [1., 1., 1.],
        [0., 0., 0.],
        [1., 1., 1.]]])

In [7]:
def number_of_channels(image):
    if image.ndim == 3:
        return image.shape[2]
    return 1

In [8]:
def extract_features(image, r=2):
    #grab the spatial dimensions of the image, along with
    # the spatial dimensions of the kernel
    (iH, iW) = image.shape[:2]
    
    kernel = window_locations(r, number_of_channels(image))
    (kH, kW) = kernel.shape[:2]

    # allocate memory for the output image, taking care to
    # "pad" the borders of the input image so the spatial
    # size (i.e., width and height) are not reduced
    pad = (kW - 1) // 2
    image = cv.copyMakeBorder(image, pad, pad, pad, pad, cv.BORDER_REPLICATE)
    output = np.zeros((iH, iW, 9 *  number_of_channels(image)), dtype="float32")
    # loop over the input image, "sliding" the kernel across
    # each (x, y)-coordinate from left-to-right and top to
    # bottom
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            # extract the ROI of the image by extracting the
            # *center* region of the current (x, y)-coordinates
            # dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]

            # perform the actual selection by taking the
            # element-wise multiplicate between the ROI and
            # the kernel
            k = (roi * kernel)

            # store the convolved value in the output (x,y)-
            # coordinate of the output image
            output[y - pad, x - pad, :] = k[np.nonzero(kernel)]
    
    # return the output image
    return output

In [9]:
from scipy import ndimage

def std_image(image):
    average = avg_image(image)
    x2_min_avg = (image - average) * (image - average);
    avg_kernel_3d = np.ones((3, 3, number_of_channels(image)), dtype=np.float32)
    avg_kernel_3d /= 9
    conv_image = ndimage.convolve(x2_min_avg, avg_kernel_3d, np.float32) 
    return np.sqrt(conv_image)

In [10]:
def normalize_image(image):
    return (image - image.min()) / (image.max() - image.min())


In [11]:
std = std_image(images[0])
import matplotlib.pyplot as plt
%matplotlib inline

std.dtype

dtype('float32')

In [12]:
def get_features(image):
    avg = avg_image(image)
    std = std_image(image)
    
    avg_output2 = extract_features(avg)
    std_output2 = extract_features(std)
    
    avg_output4 = extract_features(avg, 4)
    std_output4 = extract_features(std, 4)
    return np.concatenate((avg_output2, std_output2, avg_output4, std_output4), axis=2)

In [13]:
def get_features_for(images):
    feature_list = []
    for image in images:
        feature_list.append(get_features(image))
    return np.array(feature_list)

In [ ]:
X = get_features_for(images)
X.shape

In [ ]:
pixel_features = X.reshape(120*75*75, 108)
pixel_features.shape

In [ ]:
pixel_masks = mask_images.reshape(120*75*75)
pixel_masks.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(pixel_features, pixel_masks, test_size = 0.20)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
clf.feature_importances_

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def predict(clf, X_test, y_test):
    predictions = clf.predict(X_test)
    print(classification_report(y_test, predictions))
    print("accuracy={}".format(accuracy_score(y_test, predictions)))
    print(confusion_matrix(y_test, predictions))

In [ ]:
predict(clf, X_test, y_test)

In [ ]:
im_features = get_features_for([images[-1]])
im_features = im_features.reshape(75*75, 108)

In [ ]:
im_pred = clf.predict(im_features)
im_pred.shape

In [ ]:
prediction_image = im_pred.reshape(75, 75)
plt.imshow(prediction_image)

In [ ]:
plt.imshow(images[-1][:,:,::-1])

In [ ]:
from ProjectPaths import ProjectPaths
from Datasets import Datasets


ProjectPaths.instance(r"/home/tdjg/Documents/DeepSolaris")
ac_dataset = Datasets.datasets()["AcMüDüHo"]
train = ac_dataset[0].images

In [ ]:
plt.imshow(train[8][:,:,::-1])

In [ ]:
im_features2 = get_features_for([train[8]])
im_features2 = im_features2.reshape(75*75, 108)

im_pred = clf.predict(im_features2)
prediction_image = im_pred.reshape(75, 75)

plt.imshow(prediction_image)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, y_train)

In [ ]:
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
accuracy_score(y_test, predictions)

In [ ]:
confusion_matrix(y_test, predictions)

In [ ]:
plt.imshow(train[8][:,:,::-1])

In [ ]:
im_features3 = get_features_for([train[8]])
im_features3 = im_features3.reshape(75*75, 108)

im_pred = clf.predict(im_features3)
prediction_image = im_pred.reshape(75, 75)

plt.imshow(normalize_image(prediction_image))

# PCA

In [ ]:
import math

def reorderMatrix(u):
    c1 = np.array([1/3 * math.sqrt(3) for _ in range(3)])
    c2 = np.array([1, -1, -1] * c1)
    c3 = np.array([0, -1/2 * math.sqrt(2), 1/2 * math.sqrt(2)])
    v1 = u.dot(c2)
    i1 = np.argmax(v1)
    v2 = u.dot(c3)
    i2 = np.argmax(v2)
    if i1 == 2 and i2 == 1:
        temp = u[i1]
        u[i1] =  u[i2]
        u[i2] = temp
    return u

def flattenImage(rgbImage):
    return rgbImage.reshape(rgbImage.shape[0] * rgbImage.shape[1], 3)

def imageMean(image):
    return (1 / image.size) * np.sum(image)

def centerImage(image):
    return image - imageMean(image)

def imageCovariance(image1, image2):
    return (1 / image1.size) * np.dot(centerImage(image1).ravel(), centerImage(image2).ravel())

def rgbImageCovarianceMatrix(rgbImage):
    covarianceMatrix = np.zeros([3,3])
    for r in range(3):
        for c in range (3):
            covariance = imageCovariance(rgbImage[:,:,r], rgbImage[:,:,c])
            covarianceMatrix[r, c] = covariance
            covarianceMatrix[c, r] = covariance
    return covarianceMatrix

def pca(rgbImage):
    covarianceMatrix = rgbImageCovarianceMatrix(rgbImage)
    return np.linalg.svd(covarianceMatrix)

def pcaTransform(rgbImage):
    u,s,v = pca(rgbImage)
    imageVector = flattenImage(rgbImage)
    transposedVector = imageVector.dot(reorderMatrix(u))
    return transposedVector.reshape(rgbImage.shape)

In [ ]:
pca_image = pcaTransform(images[0])

_, ax = plt.subplots(1,4,figsize=(14,14))
ax[0].imshow(images[0])
ax[1].imshow(normalize_image(pca_image)[:,:,0])
ax[2].imshow(normalize_image(pca_image)[:,:,1])
ax[3].imshow(normalize_image(pca_image)[:,:,2])

In [ ]:
pca_images = np.array([pcaTransform(image) for image in images])
pca_images.shape

In [ ]:
pca_features = get_features_for(pca_images)
pca_pixel_features = pca_features.reshape(120*75*75, 108)
pca_pixel_features.shape

In [ ]:
pca_X_train, pca_X_test, pca_y_train, pca_y_test = train_test_split(pca_pixel_features, pixel_masks, test_size = 0.20)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(pca_X_train, pca_y_train)

In [ ]:
predict(clf, pca_X_test, pca_y_test)

In [ ]:
from sklearn.decomposition import PCA

X = images[0].reshape(75*75,3)

pca = PCA()
pca.fit(X)

In [ ]:
pca.explained_variance_

In [ ]:
im_pca = pca.transform(X).reshape(75,75,3)

_, ax = plt.subplots(1,4,figsize=(14,14))
ax[0].imshow(images[0][:,:,::-1])
ax[1].imshow(normalize_image(im_pca)[:,:,0], cmap='gray')
ax[2].imshow(normalize_image(im_pca)[:,:,1], cmap='gray')
ax[3].imshow(normalize_image(im_pca)[:,:,2], cmap='gray')

In [ ]:
def pca_transform(image):
    X = image.reshape(image.shape[0]*image.shape[1], image.shape[2])
    pca = PCA()
    pca.fit(X)
    return pca.transform(X).reshape(image.shape)

ac_pca_images = np.array([pca_transform(image) for image in ac_dataset[0].images])
ac_pca_images.shape

In [ ]:
ac_pca_features = get_features_for(ac_pca_images[:120])
ac_pca_pixel_features = ac_pca_features.reshape(120*75*75, 108)
ac_pca_pixel_features.shape

In [ ]:
ac_dataset[0].labels

In [ ]:
_, ax = plt.subplots(4,4, figsize=(14,14))

i = 0
for r in range(4):
    for c in range(4):
        ax[r,c].imshow(normalize_image(ac_pca_images[i]))
        i += 1

In [ ]:
im_pred = clf.predict(ac_pca_pixel_features)
prediction_images = im_pred.reshape(120, 75, 75)

_, ax = plt.subplots(4,4, figsize=(14,14))

i = 0
for r in range(4):
    for c in range(4):
        ax[r,c].imshow(prediction_images[i])
        i += 1

# Normalization

In [ ]:
image = images[0]
mean_per_channel = np.mean(image, axis = (0,1))    
mean_per_channel = np.ones((75,75,3)) * mean_per_channel

In [ ]:
mean_substracted_image = images[0] - mean_per_channel
mean_substracted_image 

In [ ]:
std_per_channel = np.std(image, axis = (0,1))
std_per_channel = np.ones((75,75,3)) * std_per_channel
std_per_channel

In [ ]:
normalized_image = mean_substracted_image / std_per_channel
normalized_image.min(), normalized_image.max()

In [ ]:
def z_normalize_image(image):
    mean_per_channel = np.mean(image, axis = (0,1))    
    mean_per_channel = np.ones((75,75,3)) * mean_per_channel
    mean_substracted_image = images[0] - mean_per_channel
    std_per_channel = np.std(image, axis = (0,1))
    std_per_channel = np.ones((75,75,3)) * std_per_channel
    return mean_substracted_image / std_per_channel

In [ ]:
z_image = z_normalize_image(images[0])
plt.imshow(normalize_image(z_image))

# To grayscale

In [ ]:
images.shape

In [14]:
gray_images = np.array([cv.cvtColor(image, cv.COLOR_BGR2GRAY).reshape(image.shape[0], image.shape[1], 1) for image in images])
gray_images.shape

(120, 75, 75, 1)

In [ ]:
plt.imshow(gray_images[1][:,:, 0], cmap='gray')

In [15]:
gray_features = get_features_for(gray_images)
gray_features.shape

MemoryError: 

In [ ]:
gray_pixel_features = gray_features.reshape(120*75*75, gray_features[:-1])
gray_pixel_features.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(pixel_features, pixel_masks, test_size = 0.20)

In [ ]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
predict(clf, X_test, y_test)